# To load fine-tuned model and generate test sentences

In [ ]:
# load dataset
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import time
import gc
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline as hf_pipeline
import torch

# Settings
model_id = "charlottepuopolo/sealion-3v-9b-it-taglish"
csv_path = "/content/drive/MyDrive/test-set/test-tweets-english.csv"
output_csv_path = "/content/drive/MyDrive/results/2022-tweets-ft/taglish-5-outputs.csv" # ← change as needed
batch_size = 4  # Adjust based on your GPU memory

# Load the test sentences
df = pd.read_csv(csv_path)
df.rename(columns={df.columns[0]: "english_tweet"}, inplace=True)
test_sentences = df["english_tweet"].tolist()

In [ ]:
# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Create generation pipeline
text_generator = hf_pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

In [ ]:
# GENERATE CODE SWITCHED TRANSLATIONS

def generate_code_switched_text(text, prompt_type="english"):
    # Modify prompts here
    if prompt_type == "english-0-shot":
        prompt = f"You are a machine translation system that translates sentences in the Tweet domain. Please translate this Tweet from English to bilingual code-switching of Tagalog and English. Do not output any additional text other than the translation.\n\n{text}\n\nTranslation:"
    elif prompt_type == "taglish-0-shot":
        prompt = f"Isa kang machine translation system na nagta-translate ng mga sentences sa Tweet domain. Paki translate itong Tweet from English to bilingual code-switching ng Tagalog at English. Huwag kang mag-output ng kahit anong additional text maliban sa translation.\n\n{text}\n\nTranslation:"
    elif prompt_type == "english-10-shot":
        prompt = f"""You are a machine translation system that translates sentences in the Tweet domain. Below are examples:
Example 1:
English: It seems scary to try the chicken sandwich from Jollibee hmm i saw a lot of raw parts
Taglish: parang ang skeri tuloy tikman nung chicken sandwich sa jollibee hmmm dame ko nakikitang hilaw

Example 2:
English: You're really brutal with your suggestions
Taglish: Bangis mo talaga sa mga suggestions mo

Example 3:
English: There's always that child that has the same attitude of a dad, one that is really mischievous. Huhuhu Yuri ily
Taglish: there's always that child talaga na sobrang kaugali ng tatay na sobrang ma loko huhuhu yuri ily po

Example 4:
English: Foreshadowing to the author.. I feel nervous until the end, because there's a limit...can't it just be forever? — Let's do forever next time
Taglish: Foreshadowing to author..kinakabahan ako sa hanggang dulo, kasi may hangganan eh...di ba pwedeng palagi na lang? — Sige gawin nating palagi next time

Example 5:
English: Ate Ella, i can follow right? I believe this is me because my birthday is soon 🤩
Taglish: pde naman humabol ate ella di ba? i believe ako na to dahil malapit na ang bday ko 🤩

Example 6:
English: If all of this has to do with boundaries, that BB addressed the other day, then it is management's fault. Why wasn't it addressed directly to TJ? It should have been for him anyway.
Taglish: Kung may kinalaman ang lahat ng ito sa 'boundaries' na niaddress ni BB noong isang araw, then kasalanan to ng management, bakit hindi na lang kasi direktang inaddress kay TJ total para sa kanya naman talaga dapat yun.

Example 7:
English: I don't care, whatever. You changed anyway right?
Taglish: idc, bahala ka. U changed naman na di ba?

Example 8:
English: hahahaha, fuck. Sometimes I'm really too lazy to reply. If I'm in the mood, that's the only time I'll reply. Hahaha but anyway, thank you! I appreciate your kind words for me! Love lots!!
Taglish: hahahaha, taena minsan tamad talaga ako mag reply pag asa mood ako tsaka lang ako mag rereply hahaha but anw, thank you! i appreciate your kind words for me! love lots !!

Example 9:
English: I want to finish it but fuck
Taglish: i wanna finish it pero tanhina talaga

Example 10:
English: I am actually angry. They used 875 again for views
Taglish: Galit ako actually. Ginamit na naman nila 875 para sa views

Please translate this Tweet from English to bilingual code-switching of Tagalog and English. Do not output any additional text other than the translation.
English: {text}
Taglish:
"""
    elif prompt_type == "taglish-10-shot":
        prompt = f"""Isa kang machine translation system na nagta-translate ng mga sentences sa Tweet domain. Below ang ilang halimbawa:
Halimbawa 1:
English: It seems scary to try the chicken sandwich from Jollibee hmm i saw a lot of raw parts
Taglish: parang ang skeri tuloy tikman nung chicken sandwich sa jollibee hmmm dame ko nakikitang hilaw

Halimbawa 2:
English: You're really brutal with your suggestions
Taglish: Bangis mo talaga sa mga suggestions mo

Halimbawa 3:
English: There's always that child that has the same attitude of a dad, one that is really mischievous. Huhuhu Yuri ily
Taglish: there's always that child talaga na sobrang kaugali ng tatay na sobrang ma loko huhuhu yuri ily po

Halimbawa 4:
English: Foreshadowing to the author.. I feel nervous until the end, because there's a limit...can't it just be forever? — Let's do forever next time
Taglish: Foreshadowing to author..kinakabahan ako sa hanggang dulo, kasi may hangganan eh...di ba pwedeng palagi na lang? — Sige gawin nating palagi next time

Halimbawa 5:
English: Ate Ella, i can follow right? I believe this is me because my birthday is soon 🤩
Taglish: pde naman humabol ate ella di ba? i believe ako na to dahil malapit na ang bday ko 🤩

Halimbawa 6:
English: If all of this has to do with boundaries, that BB addressed the other day, then it is management's fault. Why wasn't it addressed directly to TJ? It should have been for him anyway.
Taglish: Kung may kinalaman ang lahat ng ito sa 'boundaries' na niaddress ni BB noong isang araw, then kasalanan to ng management, bakit hindi na lang kasi direktang inaddress kay TJ total para sa kanya naman talaga dapat yun.

Halimbawa 7:
English: I don't care, whatever. You changed anyway right?
Taglish: idc, bahala ka. U changed naman na di ba?

Halimbawa 8:
English: hahahaha, fuck. Sometimes I'm really too lazy to reply. If I'm in the mood, that's the only time I'll reply. Hahaha but anyway, thank you! I appreciate your kind words for me! Love lots!!
Taglish: hahahaha, taena minsan tamad talaga ako mag reply pag asa mood ako tsaka lang ako mag rereply hahaha but anw, thank you! i appreciate your kind words for me! love lots !!

Halimbawa 9:
English: I want to finish it but fuck
Taglish: i wanna finish it pero tanhina talaga

Halimbawa 10:
English: I am actually angry. They used 875 again for views
Taglish: Galit ako actually. Ginamit na naman nila 875 para sa views

Paki translate itong Tweet from English to bilingual code-switching ng Tagalog at English. Huwag kang mag-output ng kahit anong additional text maliban sa translation.
English: {text}
Taglish:
"""
    elif prompt_type == "english-5-shot":
        prompt = f"""You are a machine translation system that translates sentences in the Tweet domain. Below are examples:
Example 1:
English: You're really brutal with your suggestions
Taglish: Bangis mo talaga sa mga suggestions mo

Example 2:
English: There's always that child that has the same attitude of a dad, one that is really mischievous. Huhuhu Yuri ily
Taglish: there's always that child talaga na sobrang kaugali ng tatay na sobrang ma loko huhuhu yuri ily po

Example 3:
English: Foreshadowing to the author.. I feel nervous until the end, because there's a limit...can't it just be forever? — Let's do forever next time
Taglish: Foreshadowing to author..kinakabahan ako sa hanggang dulo, kasi may hangganan eh...di ba pwedeng palagi na lang? — Sige gawin nating palagi next time

Example 4:
English: I don't care, whatever. You changed anyway right?
Taglish: idc, bahala ka. U changed naman na di ba?

Example 5:
English: hahahaha, fuck. Sometimes I'm really too lazy to reply. If I'm in the mood, that's the only time I'll reply. Hahaha but anyway, thank you! I appreciate your kind words for me! Love lots!!
Taglish: hahahaha, taena minsan tamad talaga ako mag reply pag asa mood ako tsaka lang ako mag rereply hahaha but anw, thank you! i appreciate your kind words for me! love lots !!

Please translate this Tweet from English to bilingual code-switching of Tagalog and English. Do not output any additional text other than the translation.
English: {text}
Taglish:
"""
    elif prompt_type == "taglish-5-shot":
        prompt = f"""Isa kang machine translation system na nagta-translate ng mga sentences sa Tweet domain. Below ang ilang halimbawa:
Halimbawa 1:
English: You're really brutal with your suggestions
Taglish: Bangis mo talaga sa mga suggestions mo

Halimbawa 2:
English: There's always that child that has the same attitude of a dad, one that is really mischievous. Huhuhu Yuri ily
Taglish: there's always that child talaga na sobrang kaugali ng tatay na sobrang ma loko huhuhu yuri ily po

Halimbawa 3:
English: Foreshadowing to the author.. I feel nervous until the end, because there's a limit...can't it just be forever? — Let's do forever next time
Taglish: Foreshadowing to author..kinakabahan ako sa hanggang dulo, kasi may hangganan eh...di ba pwedeng palagi na lang? — Sige gawin nating palagi next time

Halimbawa 4:
English: I don't care, whatever. You changed anyway right?
Taglish: idc, bahala ka. U changed naman na di ba?

Halimbawa 5:
English: hahahaha, fuck. Sometimes I'm really too lazy to reply. If I'm in the mood, that's the only time I'll reply. Hahaha but anyway, thank you! I appreciate your kind words for me! Love lots!!
Taglish: hahahaha, taena minsan tamad talaga ako mag reply pag asa mood ako tsaka lang ako mag rereply hahaha but anw, thank you! i appreciate your kind words for me! love lots !!

Paki translate itong Tweet from English to bilingual code-switching ng Tagalog at English. Huwag kang mag-output ng kahit anong additional text maliban sa translation.
English: {text}
Taglish:
"""
    else:
        print("Warning: Unknown prompt_type. Falling back to 'english'.")
        prompt = f"You are a machine translation system that translates sentences in the Tweet domain. Please translate this Tweet from English to bilingual code-switching of Tagalog and English. Do not output any additional text other than the translation.\n\n{text}\n\nTranslation:"

    messages = [{"role": "user", "content": prompt}]
    output = text_generator(prompt, max_new_tokens=100) # look at max_new_tokens if it crashes. Could modify temp and do_sample here but im not.
    return output[0]["generated_text"]


In [ ]:
# Load test sentences
df = pd.read_csv(csv_path)
df.rename(columns={df.columns[0]: "english_tweet"}, inplace=True)

# Batch processing and saving
results = []

prompt_type="taglish-5-shot" # ← change as needed

for i, row in df.iterrows():
    eng_text = row["english_tweet"]
    translated_text = generate_code_switched_text(eng_text, prompt_type=prompt_type)
    results.append((eng_text, translated_text))

    if (i + 1) % batch_size == 0:
        temp_df = pd.DataFrame(results, columns=["english_text", "code_switched_text"])
        temp_df.to_csv(output_csv_path, mode="a", header=not bool(i), index=False)
        results = []
        print(f"Saved progress at {i+1} sentences.")

    torch.cuda.empty_cache()
    gc.collect()
    time.sleep(0.5)

# Final save
final_df = pd.DataFrame(results, columns=["english_text", "code_switched_text"])
final_df.to_csv(output_csv_path, mode="a", header=False, index=False)

print(f"✅ Finished processing! Saved results to {output_csv_path}")